# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [36]:
pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 9.8 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.6-py3-none-any.whl size=138889 sha256=1747610284f8adc0732b7a75cc76e2cff89f5cc8dbc81ec52ad7391d9fec5b81
  Stored in directory: /Users/vivin/Library/Caches/pip/wheels/a6/5e/0f/8319805c4115320e0d3e8fb5799b114a2e4c4a3d6c7e523b06
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.


In [37]:
import numpy as np 
import pandas as pd 
import math 
import requests
import yfinance as yf
#import xlsxwriter

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [89]:
df = pd.read_csv('constituents.csv')
Tickers = df['Symbol']
Tickers

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [74]:
#from secrets_1 import IEX_CLOUD_API_TOKEN
tickerSymbol = 'AAPL'
tickerData = yf.Ticker(tickerSymbol)
print(tickerData)

yfinance.Ticker object <AAPL>


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [75]:
# symbol = 'AAPL'
# api_url =  f'https://sandbox.iexapis.com/stable/stock/AAPL/quote?token={IEX_CLOUD_API_TOKEN}'
# data = requests.get(api_url)
# print(data.status_code)


## Testing API call for specific fields 

The API call that we executed in the last code block seems to have all teh required data to build an Equal weighted S&P 500 fund. Testing MarketCap and price endpoints 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [79]:
marketCap = tickerData.info['marketCap']
marketCap 
previousClose = tickerData.info['previousClose']
previousClose

234.82

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [90]:
# Assuming df is your initial DataFrame containing a column 'Ticker' with the ticker symbols
#df = pd.DataFrame({'Ticker': ['AAPL', 'GOOGL', 'MSFT']})  # Example tickers

# Initialize an empty list to collect data
market_cap_data = []
# Fetch market capitalization for each ticker and append to the list
for ticker in Tickers:
    try:
        market_cap = yf.Ticker(ticker).info['marketCap']
        market_cap_data.append({'ticker': ticker, 'marketC': market_cap})
    except:
        continue
    #except Exception as e:
      # continue
# Convert the list to a DataFrame
marketCap_df = pd.DataFrame(market_cap_data)

print(marketCap_df)
    

    ticker       marketC
0      MMM   57903697920
1      AOS   13220833280
2      ABT  182104457216
3     ABBV  309504049152
4      ACN  203950637056
..     ...           ...
496    XYL   33700134912
497    YUM   36423467008
498   ZBRA   16757068800
499    ZBH   22846095360
500    ZTS   83068510208

[501 rows x 2 columns]


In [91]:
print(marketCap_df.head)

<bound method NDFrame.head of     ticker       marketC
0      MMM   57903697920
1      AOS   13220833280
2      ABT  182104457216
3     ABBV  309504049152
4      ACN  203950637056
..     ...           ...
496    XYL   33700134912
497    YUM   36423467008
498   ZBRA   16757068800
499    ZBH   22846095360
500    ZTS   83068510208

[501 rows x 2 columns]>


In [93]:
Columns_final = ['Ticker','Stock Price','Market Capitalization','Num Shares to Buy']
Output_df = pd.DataFrame(columns=Columns_final)

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [94]:
priceData =[]
for ticker in Tickers:
    try:
        market_cap = yf.Ticker(ticker).info['marketCap']
        market_cap_data.append(market_cap)
        price = yf.Ticker(ticker).info['previousClose']
        priceData.append(price)
    except:
        continue

In [101]:
Output_df['Stock Price'] = priceData
Output_df['Market Capitalization']=market_cap_data[0:501]
Output_df['Ticker'] = Tickers
Output_df

,Ticker,Stock Price,Market Capitalization,Num Shares to Buy
0,MMM,103.31,"{'ticker': 'MMM', 'marketC': 57903697920}",NaN
1,AOS,91.98,"{'ticker': 'AOS', 'marketC': 13220833280}",NaN
2,ABT,102.72,"{'ticker': 'ABT', 'marketC': 182104457216}",NaN
3,ABBV,169.20,"{'ticker': 'ABBV', 'marketC': 309504049152}",NaN
4,ACN,325.00,"{'ticker': 'ACN', 'marketC': 203950637056}",NaN
...,...,...,...,...
496,WYNN,141.26,"{'ticker': 'XYL', 'marketC': 33700134912}",NaN
497,XEL,128.92,"{'ticker': 'YUM', 'marketC': 36423467008}",NaN
498,XYL,335.76,"{'ticker': 'ZBRA', 'marketC': 16757068800}",NaN
499,YUM,107.46,"{'ticker': 'ZBH', 'marketC': 22846095360}",NaN


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: